<br />Created on August 25, 2024 at 23:59 am.
<br />@author: zhaolx
<br />Revised on August 26, 2024 at 11:53 pm.

# Code to SSH (sunshine hours)

In [ ]:
import math
import pandas as pd
# The latitude of Tianjing is 39 degrees
phi = 41.75  # in degrees
phi_rad = math.radians(phi)  # convert to radians
G0 = 1367  # Solar constant in W/m^2

# Integrand function for spatial integral
def integrand(x, y, c, a, h, eta):
    term1 = c * y * np.cosh(math.radians(h))
    term2 = a * np.sqrt(a**2 - (x**2 + y**2))
    term3 = term1 + term2
    term4 = np.sqrt(1 + a**2 / (a**2 - (x**2 + y**2)))
    return (term3 / a) * term4 * eta

# Function to calculate delta based on n
def calculate_delta(n):
    delta = math.degrees(math.asin(math.sin(math.radians(-23.44)) * math.cos(math.radians((360 / 365) * (n + 10)))))
    return delta

# Function to calculate omega based on t
def calculate_omega(t):
    omega = 15 * (t - 12)
    return omega

# Function to calculate h based on delta and omega
def calculate_h(delta, omega):
    delta_rad = math.radians(delta)  # convert delta to radians
    omega_rad = math.radians(omega)  # convert omega to radians
    h = math.degrees(math.asin(math.sin(delta_rad) * math.sin(phi_rad) +
                               math.cos(delta_rad) * math.cos(phi_rad) * math.cos(omega_rad)))
    return h

# Function to calculate G
def calculate_G(G0, n, t):
    delta = calculate_delta(n)
    omega = calculate_omega(t)
    h = calculate_h(delta, omega)
    G = G0 * (1 + 0.033 * math.cos((n / 365) * 2 * math.pi)) * math.cos(math.radians(h))
    return G

# Function to find time when h is closest to zero
# Use bisection to find the time when h is closest to 0
def find_sunrise_sunset(n, step=0.001):
    delta = calculate_delta(n)
    closest_t_sunrise = None
    closest_h_sunrise = float('inf')
    for t in [i * step for i in range(int(12 / step))]:
        omega = calculate_omega(t)
        h = calculate_h(delta, omega)
        if abs(h) < abs(closest_h_sunrise):
            closest_h_sunrise = h
            closest_t_sunrise = t
    closest_t_sunset = None
    closest_h_sunset = float('inf')
    for t in [i * step for i in range(int(12 / step), int(24 / step))]:
        omega = calculate_omega(t)
        h = calculate_h(delta, omega)
        if abs(h) < abs(closest_h_sunset):
            closest_h_sunset = h
            closest_t_sunset = t
    return closest_t_sunrise, closest_h_sunrise, closest_t_sunset, closest_h_sunset

In [ ]:
# Example1: Calculate h for n=1 and t from 0 to 24
n = 1
delta = calculate_delta(n)
h_values = []
for t in range(25):  # t ranges from 0 to 24 inclusive
    omega = calculate_omega(t)
    h = calculate_h(delta, omega)
    h_values.append((t, h))
# Filter for positive h values
positive_h_times = [t for t, h in h_values if h > 0]
positive_h_times

In [ ]:
# Example2: Find sunrise and sunset for day n=1
n = 1
sunrise_t, sunrise_h, sunset_t, sunset_h = find_sunrise_sunset(n)
print(f"Sunrise time: {sunrise_t:.4f} hours, h: {sunrise_h:.6f} degrees")
print(f"Sunset time: {sunset_t:.4f} hours, h: {sunset_h:.6f} degrees")
daylight_duration = sunset_t - sunrise_t
daylight_duration_rounded = round(daylight_duration, 2)
print(f"Daylight duration: {daylight_duration_rounded} hours")

In [ ]:
# Leap Year (366 days)
# Noleap Year (365 days)
import csv
# File path to save the results
file_path = "/SHENYANG_daylight_time_NoLeapYear.csv"
# Open the file for writing
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['n', 'sunrise_t', 'sunset_t', 'daylight_duration'])
    # Loop through each day of the year
    for n in range(1, 366):
        sunrise_t, sunrise_h, sunset_t, sunset_h = find_sunrise_sunset(n)
        daylight_duration = sunset_t - sunrise_t
        daylight_duration_rounded = round(daylight_duration, 2)
        writer.writerow([n, f"{sunrise_t:.4f}", f"{sunset_t:.4f}", daylight_duration_rounded])
print("Results saved to", file_path)

In [ ]:
# Perform calculations
# Calculate h for n=1 and t=7.9
n = 1
t = 7.9
delta = calculate_delta(n)
omega = calculate_omega(t)
h = calculate_h(delta, omega)

# Print the result
print(f"At n={n}, t={t} hours, the solar elevation angle h is: {h:.6f} degrees")

In [ ]:
import matplotlib.pyplot as plt
# Parameters
n = 1  # Day of the year
delta = calculate_delta(n)

# Calculate h values for t from 0 to 24
t_values = range(25)  # t ranges from 0 to 24 inclusive
h_values = [calculate_h(delta, calculate_omega(t)) for t in t_values]

# Plot h vs. t
plt.figure(figsize=(10, 6))
plt.plot(t_values, h_values, marker='o')
plt.axhline(y=0, color='r', linestyle='--', label='Horizon (h = 0)')
plt.xlabel('Time (t) in hours')
plt.ylabel('Solar Elevation Angle (h) in degrees')
plt.title('Solar Elevation Angle (h) vs. Time (t) for n=1')
plt.grid(True)
plt.legend()
plt.xticks(range(0, 25, 1))  # Ensure x-axis shows each hour
plt.show()

# Code to calculate PET(potential_evapotranspiration)


<br /> Thornthwaite method is used to calculate PET.
<br /> Only calculate monthly PET

## formulas 1: Thornthwaite method
<br /> monthly PET

In [ ]:
import pandas as pd
import numpy as np
## Part1
# Step 1: Read data
df = pd.read_csv('/54342099999_2004.csv')
# Replace erroneous temperature values with NaN
df[df['TEMP'] < 0] = np.nan
ab = df.dropna(axis=0, how='all')
# Step 2: Convert temperature from Fahrenheit to Celsius
ab['TEMP_C'] = (ab['TEMP'] - 32) * 5 / 9
# Step 3: Add DATE and extract Month
ab['DATE'] = pd.to_datetime(ab['DATE'])  # Ensure DATE is in datetime format
ab['Month'] = ab['DATE'].dt.month  # Extract the month
# Step 4: Calculate the monthly average of TEMP_C
Sum_table = ab.groupby('Month')['TEMP_C'].mean().reset_index(name='TEMP_C_Monthly_avg')
# Step 5: Calculate T using the formula
Sum_table['Hi'] = np.power(Sum_table['TEMP_C_Monthly_avg'] / 5, 1.514)
print(Sum_table)
# Save the result to CSV
Sum_table.to_csv('/SHENYANG_monthly_T.csv', index=False)
print("------------------------")
## Part2
# Read the CSV file
df_sum = pd.read_csv('/SHENYANG_monthly_T.csv')
# Extract TEMP_C_Monthly_avg values
T = df_sum['TEMP_C_Monthly_avg'].values
# Calculate H_sum as the sum of the Hi column
H_sum = df_sum['Hi'].sum()
# Calculate the coefficient A
A = (6.75e-07)*np.power(H_sum, 3) - (7.71e-05)*np.power(H_sum, 2) + (1.792e-02)*H_sum + 0.49
# Calculate PET and replace NaN values with 0
PET = 16 * np.power((10 * T) / H_sum, A)
PET = np.nan_to_num(PET)  # Replace NaN with 0
# Add PET to the DataFrame
df_sum['PET'] = PET
# Save the result to CSV
df_sum.to_csv('/SHENYANG_monthly_PET.csv', index=False)
print(df_sum)

## formulas 1: Thornthwaite method
<br /> daily PET

In [ ]:
import pandas as pd
import numpy as np
# Read data
df = pd.read_csv('/54342099999_2004.csv')
# Replace temperatures below 0 with NaN
df.loc[df['TEMP'] < 0, 'TEMP'] = np.nan
# Drop rows where all elements are NaN
df_cleaned = df.dropna(axis=0, how='all')
# Convert temperature to Celsius
df_cleaned['TEMP_C'] = (df_cleaned['TEMP'] - 32) * 5 / 9
# Ensure DATE is in datetime format and extract the month
df_cleaned['DATE'] = pd.to_datetime(df_cleaned['DATE'])
df_cleaned['Month'] = df_cleaned['DATE'].dt.month
# Calculate 'Hi' values and create a summary table
df_cleaned['Hi'] = np.power(df_cleaned['TEMP_C'] / 5, 1.514)
Sum_table = df_cleaned[['Month', 'Hi']].groupby('Month').agg({'Hi': 'sum'})
# Calculate A for each month based on H_sum
Sum_table['A'] = (6.75e-07) * np.power(Sum_table['Hi'], 3) - (7.71e-05) * np.power(Sum_table['Hi'], 2) + (1.792e-02) * Sum_table['Hi'] + 0.49

# Merge the H_sum and A back to the main dataframe
df_cleaned = df_cleaned.merge(Sum_table, on='Month', how='left', suffixes=('', '_monthly'))

# Calculate PET
df_cleaned['PET'] = np.where(df_cleaned['TEMP_C'] < 0, 
                             0, 
                             16 * np.power((10 * df_cleaned['TEMP_C']) / df_cleaned['Hi_monthly'], df_cleaned['A']))

# Optionally, inspect the result
print(df_cleaned[['DATE', 'TEMP_C', 'PET']].head())
# Save the desired columns to a CSV file
df_cleaned[['DATE', 'TEMP_C', 'PET']].to_csv('/SHENYANG_daily_PET.csv', index=False)


## formulas 2: Thornthwaite method considering the SSH

$PET=\begin{cases}0&T<0\\16(\frac{N}{12})(\frac{NDM}{30})(\frac{10T}{1})^m&0\le T<26.5\\-415.85+32.24T-0.43T^2&T\ge26.5\end{cases}$

In [ ]:
# Read the CSV file
df_sum = pd.read_csv('/SHENYANG_monthly_T.csv')
SSH_LEAP = pd.read_csv('/SHENYANG_daylight_time_LeapYear.csv')
# Extract TEMP_C_Monthly_avg values
T = df_sum['TEMP_C_Monthly_avg'].values
SSH = SSH_LEAP['daylight_duration'].values
# Add a column for the date using n (day of year) and the year 2004
SSH_LEAP['DATE'] = pd.to_datetime('2004-01-01') + pd.to_timedelta(SSH_LEAP['n'] - 1, unit='D')
# Extract the month from the DATE column
SSH_LEAP['Month'] = SSH_LEAP['DATE'].dt.month
# Calculate the maximum daylight duration for each month
monthly_max_SSH = SSH_LEAP.groupby('Month')['daylight_duration'].max().values
# Calculate the number of days per month (NDM) for 2004
days_per_month = SSH_LEAP.groupby('Month')['DATE'].nunique().values
# Calculate H_sum as the sum of the Hi column
H_sum = df_sum['Hi'].sum()
# Calculate the coefficient A
A = (6.75e-07)*np.power(H_sum, 3) - (7.71e-05)*np.power(H_sum, 2) + (1.792e-02)*H_sum + 0.49
# Calculate PET and replace NaN values with 0
PET = 16 *(monthly_max_SSH / 12) * (days_per_month / 30) * np.power((10 * T) / H_sum, A)
PET = np.nan_to_num(PET)  # Replace NaN with 0
# Add PET to the DataFrame
df_sum['PET'] = PET
# Save the result to CSV
df_sum.to_csv('/SHENYANG_monthly_PET2.csv', index=False)
print(df_sum)

In [ ]:
import pandas as pd
import numpy as np
## Part1
# Step 1: Read data
df = pd.read_excel('/meteorology_Shenyang_merged_data.xlsx')
# Replace erroneous temperature values with NaN
df.loc[df['TEMP'] < 0, 'TEMP'] = np.nan

# Drop rows where all values are NaN
ab = df.dropna(axis=0, how='all')

# Step 2: Convert temperature from Fahrenheit to Celsius
ab['TEMP_C'] = (ab['TEMP'] - 32) * 5 / 9

# Step 3: Parse DATE with two possible formats
# First try to parse as "YYYY-MM-DD"
ab['DATE'] = pd.to_datetime(ab['DATE'], format='%Y-%m-%d', errors='coerce')

# For rows where the date parsing failed (NaT), try "YYYY/M/D"
ab.loc[ab['DATE'].isna(), 'DATE'] = pd.to_datetime(ab.loc[ab['DATE'].isna(), 'DATE'], format='%Y/%m/%d', errors='coerce')

# Step 4: Remove rows with invalid dates
ab = ab.dropna(subset=['DATE'])

# Extract Year and Month from the parsed DATE column
ab['Year'] = ab['DATE'].dt.year
ab['Month'] = ab['DATE'].dt.month

# Step 5: Calculate the monthly average of TEMP_C grouped by Year and Month
Sum_table = ab.groupby(['Year', 'Month'])['TEMP_C'].mean().reset_index(name='TEMP_C_Monthly_avg')

# Step 6: Calculate Hi using the formula
Sum_table['Hi'] = np.power(Sum_table['TEMP_C_Monthly_avg'] / 5, 1.514)

# Print the result
print(Sum_table)
# Save the result to CSV with Year and Month preserved
Sum_table.to_csv('/SHENYANG_monthly_T.csv', index=False)

## gma package

In [ ]:
from gma import climet
import pandas as pd
df = pd.read_csv('/SHENYANG_monthly_T.csv')
# Step 2: 获取 TEMP_C_Monthly_avg 列
TEM = df['TEMP_C_Monthly_avg'].tolist()
# TEM = [0.95, 2.15, 5.75, 13, 19.95, 23.7, 24.8, 23.9, 20.15, 15.2, 10.6, 3.4,
#        0.0, 3.8, 9.85, 14.95, 20.7, 25.05, 26.55, 24.5, 19.55, 12.65, 5.9, 2.4]

THD = climet.ET0.Thornthwaite(TEM, LAT = 41.75, StartYear = 1990)
print(THD.shape)
# Step 4: 将 THD 转换为 DataFrame
thd_df = pd.DataFrame({'THD': THD})

# Step 5: 保存到 Excel 文件
output_path = '/THD_result.xlsx'
thd_df.to_excel(output_path, index=False)

print(f"THD 已保存到: {output_path}")

In [ ]:
import pandas as pd

# Step 1: 读取 THD 数据
thd_df = pd.read_excel('/SHENYANG_THD_result.xlsx')

# Step 2: 读取 Runoff_SHENYANG.xlsx 文件中的日期信息
runoff_df = pd.read_excel('/meteorology_Shenyang_merged_data.xlsx')
# Step 3: 提取日期中的年和月
runoff_df['DATE'] = pd.to_datetime(runoff_df['DATE'])
runoff_df['Year'] = runoff_df['DATE'].dt.year
runoff_df['Month'] = runoff_df['DATE'].dt.month

# Step 4: 计算每个月的天数
runoff_df['Days_in_Month'] = runoff_df['DATE'].apply(lambda x: pd.Timestamp(year=x.year, month=x.month, day=1).days_in_month)

# Step 5: 合并 THD 数据到 runoff_df 中
runoff_df = pd.merge(runoff_df, thd_df, how='left', on=['Year', 'Month'])

# Step 6: 将 THD 的月总值分摊到每天的数据上
runoff_df['THD'] = runoff_df['THD'] / runoff_df['Days_in_Month']

# Step 7: 将 THD 的 NaN 值填充为 0（如果某些月份没有 THD 值）
runoff_df['THD'].fillna(0, inplace=True)

# Step 8: 保存结果到新的 Excel 文件
output_path = '/SHENYANG_with_THD.xlsx'
runoff_df.to_excel(output_path, index=False)

print(f"已保存文件至: {output_path}")



In [ ]:
import matplotlib.pyplot as plt
import gma
PAR = {'font.sans-serif': 'Times New Roman',
       'axes.unicode_minus': False,
      }
plt.rcParams.update(PAR)

X = gma.osf.DateSeries('198101','198301',DateDelta='M', Format='%Y%m').strftime('%Y-%m')

plt.figure(figsize = (10, 5), dpi = 300)

### 绘制数据
plt.plot(X, THD, linewidth = 0.8, c = 'gray')
plt.xticks(X[::2])

### 绘制其他网格
plt.grid(True, linestyle = (0,(6,6)), linewidth = 0.3)

plt.xlabel('Date')
plt.ylabel('Thornthwaite PET(mm)')
plt.show()

## formulas 2: Hargreaves method
<br /> $ET_{0}=c_{0}R_{\mathrm{a}}(T_{\mathrm{mean}}+17.8)\sqrt{T_{\mathrm{max}}-T_{\mathrm{min}}}$

In [ ]:
from gma import climet
import numpy as np
np.random.seed(0)

TMAX = np.random.uniform(20, 30, size = 48)
TMIN = np.random.uniform(10, 20, size = 48)

THD = climet.ET0.Hargreaves(TMAX, TMIN)
import matplotlib.pyplot as plt
import gma
PAR = {'font.sans-serif': 'Times New Roman',
       'axes.unicode_minus': False,
      }
plt.rcParams.update(PAR)

X = gma.osf.DateSeries('198101','198501',DateDelta='M', Format='%Y%m').strftime('%Y-%m')

plt.figure(figsize = (10, 5), dpi = 300)

### 绘制数据
plt.plot(X, THD, linewidth = 0.8, c = 'gray')
plt.xticks(X[::2])

### 绘制其他网格
plt.grid(True, linestyle = (0,(6,6)), linewidth = 0.3)

plt.xlabel('Date')
plt.ylabel('Thornthwaite PET(mm)')
plt.show()

## formulas 3: Penman-Monteith method

In [ ]:
import gma.climet.ET0

help(gma.climet.ET0.PenmanMonteith)
#  PRS：float||array。 日平均气压（hPa）。

#  WIN：float||array 。 日平均10m风速（m/s）。

#  TMax：float||array。 日最高气温（℃）。

#  TMin：float||array 。日最低气温（℃）。

#  RHU：float||array 。日平均相对湿度（%）。

#  SSH：float||array 日日照时数（h）。

#  LAT：float||array 。纬度（°）。

#  Day：int||array 。日期，以日序表示。1-365（平年）或366（闰年）。

#  ELE：float||array 。海拔高度（m）。

In [ ]:
from gma import climet
PRS, WIN, TMAX, TMIN, RHU, SSH = 1025.3, 2.2, 5.1, -4.5, 6.3, 5.5
LAT, Day, ELE = 35.6, 350, 45
climet.ET0.PenmanMonteith(PRS, WIN, TMAX, TMIN, RHU, SSH, LAT, Day, ELE)

## Accuracy results

In [ ]:
import pandas as pd
import numpy as np
from gma import climet
df = pd.read_csv('/54342099999_2004.csv')
PRS = df['SLP'].values
WIN = df['WDSP'].values * 0.514444
max_temp = df['MAX'].values
min_temp = df['MIN'].values
TMAX = (max_temp - 32) * 5 / 9
TMIN = (min_temp - 32) * 5 / 9
Prcp = df['PRCP'].values
# 将降水量从英寸转换为毫米
Prcp_mm = Prcp * 25.4
# 转换华氏度到摄氏度
def fahrenheit_to_celsius(f):
    return (f - 32) / 1.8
# 计算饱和水汽压
def saturation_vapor_pressure(T_c):
    return 611.2 * np.exp((17.67 * T_c) / (T_c + 243.5))
# 计算相对湿度
def relative_humidity(T_d_f, T_f):
    T_d_c = fahrenheit_to_celsius(T_d_f)
    T_c = fahrenheit_to_celsius(T_f)
    
    E_s = saturation_vapor_pressure(T_d_c)
    E_a = saturation_vapor_pressure(T_c)
    
    return (E_s / E_a) * 100
# 计算相对湿度
dewp_f = df['DEWP'].values
temp_f = df['TEMP'].values
Tmean = (temp_f - 32) * 5 / 9
RHU = relative_humidity(dewp_f, temp_f)
leapyear = pd.read_csv('/SHENYANG_daylight_time_LeapYear.csv')
SSH = leapyear['daylight_duration'].values
LAT = df['LATITUDE'].values
DAY = leapyear['n'].values
ELE = df['ELEVATION'].values
PM_PET = climet.ET0.PenmanMonteith(PRS, WIN, TMAX, TMIN, RHU, SSH, LAT, DAY, ELE)
Har_PET = climet.ET0.Hargreaves(TMAX, TMIN)
# 将 numpy.ndarray 转换为 pandas.DataFrame
df_pm_pet = pd.DataFrame(PM_PET, columns=['PM_PET'])
df_har_pet = pd.DataFrame(Har_PET, columns=['Har_PET'])
df_Tmean = pd.DataFrame(Tmean, columns=['Tmean'])
df_Prcp_mm = pd.DataFrame(Prcp_mm, columns=['Prcp_mm'])
# 将 numpy.ndarray 转换为 pandas.DataFrame
df_all = pd.DataFrame({
    'Tmean': Tmean,
    'Prcp_mm': Prcp_mm,
    'PM_PET': PM_PET,
    'Har_PET': Har_PET
})

# 保存到 CSV 文件
df_all.to_csv('/SHENYANG_daily_PET_PMmethod.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from gma import climet
import os
# 定义转换函数
def fahrenheit_to_celsius(f):
    return (f - 32) / 1.8
def saturation_vapor_pressure(T_c):
    return 611.2 * np.exp((17.67 * T_c) / (T_c + 243.5))
def relative_humidity(T_d_f, T_f):
    T_d_c = fahrenheit_to_celsius(T_d_f)
    T_c = fahrenheit_to_celsius(T_f)
    E_s = saturation_vapor_pressure(T_d_c)
    E_a = saturation_vapor_pressure(T_c)
    return (E_s / E_a) * 100
# 判断是否为闰年
def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
# 定义目录路径
directory = '/SHENYANG/'
# 遍历目录中的所有文件
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # 获取年份
        year = int(filename.split('_')[-1].split('.')[0])
        # 根据年份选择相应的昼夜时长数据文件
        if is_leap_year(year):
            leapyear = pd.read_csv('/SHENYANG_daylight_time_LeapYear.csv')
        else:
            leapyear = pd.read_csv('/SHENYANG_daylight_time_NoLeapYear.csv')
        SSH = leapyear['daylight_duration'].values
        DAY = leapyear['n'].values
        # 读取数据
        df = pd.read_csv(os.path.join(directory, filename))
        # 提取数据并进行单位转换
        PRS = df['SLP'].values
        WIN = df['WDSP'].values * 0.514444
        max_temp = df['MAX'].values
        min_temp = df['MIN'].values
        TMAX = (max_temp - 32) * 5 / 9
        TMIN = (min_temp - 32) * 5 / 9
        Prcp = df['PRCP'].values
        Prcp_mm = Prcp * 25.4
        # 计算相对湿度
        dewp_f = df['DEWP'].values
        temp_f = df['TEMP'].values
        Tmean = (temp_f - 32) * 5 / 9
        RHU = relative_humidity(dewp_f, temp_f)
        # 提取纬度和海拔信息
        LAT = df['LATITUDE'].values
        ELE = df['ELEVATION'].values
        # 计算 PET
        PM_PET = climet.ET0.PenmanMonteith(PRS, WIN, TMAX, TMIN, RHU, SSH, LAT, DAY, ELE)
        Har_PET = climet.ET0.Hargreaves(TMAX, TMIN)
        # 创建 DataFrame 并保存到 CSV 文件
        df_all = pd.DataFrame({
            'DAY': DAY,
            'Tmean': Tmean,
            'Prcp_mm': Prcp_mm,
            'PM_PET': PM_PET,
            'Har_PET': Har_PET
        })
        # 保存文件
        output_filename = f'/SHENYANG_daily_PET_{year}.csv'
        df_all.to_csv(output_filename, index=False)
        print(f'Processed and saved: {output_filename}')


In [ ]:
import pandas as pd

# 读取数据
file_path = "/54342099999_2023.csv"
df = pd.read_csv(file_path)

# 将 DATE 列转换为 datetime 格式
df['DATE'] = pd.to_datetime(df['DATE'])

# 生成 1998 年的完整日期范围
full_date_range = pd.date_range(start="2023-01-01", end="2023-12-31")

# 找出缺失的日期
missing_dates = full_date_range.difference(df['DATE'])

# 打印缺失的日期
print("缺失的日期:", missing_dates)


In [ ]:
import pandas as pd
import numpy as np
from gma import climet
import os

# 定义转换函数
def fahrenheit_to_celsius(f):
    return (f - 32) / 1.8

def saturation_vapor_pressure(T_c):
    return 611.2 * np.exp((17.67 * T_c) / (T_c + 243.5))

def relative_humidity(T_d_f, T_f):
    T_d_c = fahrenheit_to_celsius(T_d_f)
    T_c = fahrenheit_to_celsius(T_f)
    
    E_s = saturation_vapor_pressure(T_d_c)
    E_a = saturation_vapor_pressure(T_c)
    
    return (E_s / E_a) * 100

# 读取数据
file_path = "/54342099999_2023.csv"
df = pd.read_csv(file_path)

# 将 DATE 列转换为 datetime 格式
df['DATE'] = pd.to_datetime(df['DATE'])

# 生成 2023 年的完整日期范围
full_date_range = pd.date_range(start="2023-01-01", end="2023-12-31")

# 找出缺失的日期
missing_dates = full_date_range.difference(df['DATE'])

# 读取 leapyear 数据
leapyear = pd.read_csv('/SHENYANG_daylight_time_NoLeapYear.csv')

# 将缺失的日期转换为日序 n
missing_days_of_year = missing_dates.day_of_year

# 剔除 leapyear 中与缺失日序相同的行
leapyear = leapyear[~leapyear['n'].isin(missing_days_of_year)]

# 提取修正后的变量
SSH = leapyear['daylight_duration'].values
DAY = leapyear['n'].values[:len(SSH)]  # 确保 DAY 的长度与修正后的 SSH 一致

# 提取其他变量
PRS = df['SLP'].values
WIN = df['WDSP'].values * 0.514444
max_temp = df['MAX'].values
min_temp = df['MIN'].values
TMAX = (max_temp - 32) * 5 / 9
TMIN = (min_temp - 32) * 5 / 9
Prcp = df['PRCP'].values
Prcp_mm = Prcp * 25.4
dewp_f = df['DEWP'].values
temp_f = df['TEMP'].values
Tmean = (temp_f - 32) * 5 / 9
RHU = relative_humidity(dewp_f, temp_f)
LAT = df['LATITUDE'].values
ELE = df['ELEVATION'].values

# 计算 PET
PM_PET = climet.ET0.PenmanMonteith(PRS, WIN, TMAX, TMIN, RHU, SSH, LAT, DAY, ELE)
Har_PET = climet.ET0.Hargreaves(TMAX, TMIN)

# 创建 DataFrame 并保存到 CSV 文件
df_all = pd.DataFrame({
    'DAY': DAY,
    'Tmean': Tmean,
    'Prcp_mm': Prcp_mm,
    'PM_PET': PM_PET,
    'Har_PET': Har_PET
})

output_filename = f'/SHENYANG_daily_PET_2023.csv'
df_all.to_csv(output_filename, index=False)

print(f'Processed and saved: {output_filename}')

